In [1]:
import openmc
import numpy as np

In [2]:
# Material
fuel = openmc.Material(name='fuel')# 建立燃料对象
fuel.set_density(units="atom/b-cm",density=8.0175E-2)# 设置燃料密度
fuel.add_nuclide("U233",1.5621E-3,percent_type="ao")# 添加核素U-233，含量占比为原子数比例
fuel.add_nuclide("Th232",1.2639E-2,percent_type="ao")# 添加核素Th-232，含量占比为原子数比例
fuel.add_nuclide("O16",2.8402E-2,percent_type="ao")# 添加核素O-16，含量占比为原子数比例
fuel.temperature = 1500 # 设置燃料温度，单位为Kelvin

clad = openmc.Material(name='clad')
clad.set_density(units='atom/b-cm',density=4.3346E-2)
clad.add_nuclide("Zr91",1,percent_type="ao")
clad.temperature = 600

moderator = openmc.Material(name="moderator")
moderator.set_density(units="atom/b-cm",density=6.5491E-2)
moderator.add_nuclide("H2",4.3661E-2,percent_type="ao")
moderator.add_nuclide("O16",2.1830E-2,percent_type="ao")
moderator.temperature = 600
moderator.add_s_alpha_beta('c_D_in_D2O') # 添加水中氘的热散射截面

materials = openmc.Materials([fuel, clad, moderator])

In [22]:
fuel

Material
	ID             =	1
	Name           =	fuel
	Temperature    =	1500
	Density        =	0.080175 [atom/b-cm]
	S(a,b) Tables  
	Nuclides       
	U233           =	0.0015621    [ao]
	Th232          =	0.012639     [ao]
	O16            =	0.028402     [ao]

In [3]:
# Geometry parameters
fuel_r = [0.52273,0.57273] # 燃料栅元半径 （忽略气隙）
cr_r = [0.5042,0.5461]# 控制棒栅元半径
gt_r = [0.43688,0.48387,0.56134,0.60198]# 导向管栅元半径
pitch = 1.26 # 栅距
assembly_len = 21.42 # 组件边长
axial_height = 200 # 组件高度

In [4]:
# 栅元快速建模方法
# fuel pin
fuel_pin_surfaces = [openmc.ZCylinder(r=r) for r in fuel_r] # 创建圆柱面列表，分别代表燃料
# 利用圆柱面列表和材料列表快速创建燃料栅元universe（省略创建cell的过程）
fuel_pin_univ = openmc.model.pin(fuel_pin_surfaces, openmc.Materials([fuel,clad,moderator])) 

# control rod pin
cr_pin_surfaces = [openmc.ZCylinder(r=r) for r in cr_r]
cr_pin_univ = openmc.model.pin(cr_pin_surfaces, openmc.Materials([moderator,clad,moderator]))

# guide tube pin
gt_pin_surfaces = [openmc.ZCylinder(r=r) for r in gt_r]
gt_pin_univ = openmc.model.pin(gt_pin_surfaces, openmc.Materials([moderator,clad,moderator,clad,moderator]))

In [5]:
# 利用lattice功能快速创建组件
# assembly lattice
lattice = openmc.RectLattice() # 创建方形组件排列
lattice.lower_left = (-assembly_len/2, -assembly_len/2) # 给定组件左下角坐标
lattice.pitch = (pitch, pitch) # 给定组件两个方向的栅距
lat = np.tile(fuel_pin_univ, (17, 17)) # 利用numpy的tile函数快速创建17*17的数组，每一个元素皆为燃料栅元

In [8]:
# 将对应位置修改成我们想要的栅元
# guide tube
lat[8,8] = gt_pin_univ
# control rod
lat[2,[5,8,11]]=cr_pin_univ
lat[14,[5,8,11]]=cr_pin_univ
lat[3,[3,13]]=cr_pin_univ
lat[13,[3,13]]=cr_pin_univ
lat[5,[2,5,8,11,14]]=cr_pin_univ
lat[11,[2,5,8,11,14]]=cr_pin_univ
lat[8,[2,5,11,14]]=cr_pin_univ

# 将组件universe填充上上述排列好的数组
lattice.universes = lat

In [23]:
lattice

RectLattice
	ID             =	4
	Name           =	
	Shape          =	(17, 17)
	Lower Left     =	(-10.71, -10.71)
	Pitch          =	(1.26, 1.26)
	Outer          =	None
	Universes      
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 2 1 1 2 1 1 2 1 1 1 1 1 
1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 2 1 1 2 1 1 3 1 1 2 1 1 2 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 
1 1 1 1 1 2 1 1 2 1 1 2 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [9]:
# root cell and universe
root_cell = openmc.Cell(name='root cell')
root_cell.fill = lattice
# 创建一个长方体区域，将组件包围
assembly_top = openmc.ZPlane(z0 = axial_height/2,boundary_type='reflective')
assembly_bottom = openmc.ZPlane(z0 = -axial_height/2,boundary_type='reflective')
bound_box = openmc.rectangular_prism(assembly_len, assembly_len, boundary_type="reflective")
root_cell.region = bound_box
root_cell.region = root_cell.region & -assembly_top & +assembly_bottom

root_univ = openmc.Universe(cells=[root_cell])
geometry = openmc.Geometry(root_univ)

In [10]:
# 设置计算参数
# settings
settings = openmc.Settings() # 创建参数对象
settings.particles = 10000 # 粒子数
settings.inactive = 100 # 非活跃代
settings.batches = 200 # 总代数
settings.temperature['multipole']= True # 调用Windowed Multipole做多普勒展开
settings.temperature['method']= 'interpolation' 

In [11]:
# 绘制几何结构
# plot
plot_xy = openmc.Plot(plot_id=1)
plot_xy.basis = 'xy'
plot_xy.filename = 'materials-xy-Height'
plot_xy.origin = [0, 0,0]
plot_xy.pixels = [3000, 3000]
plot_xy.width = (25, 25)
plot_xy.color_by = 'material'

In [13]:
# plot
plot_xz = openmc.Plot(plot_id=2)
plot_xz.basis = 'xz'
plot_xz.filename = 'materials-yz-Height'
plot_xz.origin = [0, 0,0]
plot_xz.pixels = [3000, 3000]
plot_xz.width = (100, 100)
plot_xz.color_by = 'material'

In [14]:
# 输出上述几何、计算设置、材料、几何绘制文件
# export to xml
geometry.export_to_xml()
settings.export_to_xml()
materials.export_to_xml()
plot_file = openmc.Plots([plot_xy,plot_xz])
plot_file.export_to_xml()

In [15]:
# 绘制几何
openmc.plot_geometry(output=False)

In [24]:
# 运行模拟
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

      168/1    1.18430    1.15147 +/- 0.00164
      169/1    1.17122    1.15176 +/- 0.00164
      170/1    1.15499    1.15180 +/- 0.00162
      171/1    1.15346    1.15183 +/- 0.00160
      172/1    1.13959    1.15166 +/- 0.00158
      173/1    1.14343    1.15154 +/- 0.00157
      174/1    1.16654    1.15175 +/- 0.00156
      175/1    1.15683    1.15181 +/- 0.00154
      176/1    1.14639    1.15174 +/- 0.00152
      177/1    1.17888    1.15210 +/- 0.00154
      178/1    1.14524    1.15201 +/- 0.00152
      179/1    1.15980    1.15211 +/- 0.00151
      180/1    1.15081    1.15209 +/- 0.00149
      181/1    1.18040    1.15244 +/- 0.00151
      182/1    1.15824    1.15251 +/- 0.00149
      183/1    1.16664    1.15268 +/- 0.00149
      184/1    1.14607    1.15260 +/- 0.00147
      185/1    1.16223    1.15271 +/- 0.00146
      186/1    1.15352    1.15272 +/- 0.00144
      187/1    1.15948    1.15280 +/- 0.00143
      188/1    1.14241    1.15268 +/- 0.00141
      189/1    1.13850    1.15252 